In [1]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.19.5', '1.1.5')

# Chipotle Data (식당 데이터) 분석
> 참고 블로그 : https://tjansry354.tistory.com/6

* order_id : 주문번호

* quantity : 주문 개수

* item_name : 음식 이름

* choice_description : 선택 옵션 (소스는 무엇인지 재료는 어떤 걸 골랐는지)

* item_price : 음식 가격

In [2]:
# data 불러오기
data = pd.read_csv('drive/MyDrive/Deeplearning/Python/chipotle.csv')
data

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


In [3]:
data.info()
# choice_description 컬럼에 nan값 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


## 문제 1

### 1. 음식 메뉴는 총 몇가지 일까?

In [4]:
# unique() : 중복을 제거한 값들을 보여줌
items = data.item_name.unique()

print("음식 메뉴는 총 {}개 이다".format(len(items)))

음식 메뉴는 총 50개 이다


### 메뉴 당 가격은 얼마일까? 뭐가 제일 비싸지?

In [5]:
data['item_price'] = data['item_price'].str.lstrip('$').astype(float)
# pd.to_numeric()사용하는 방법
# data['item_price'] = pd.to_numeric(data['item_price'].str.lstrip('$'))
# column 이름 바꾸기
data.rename(columns={'item_price':'item_price($)'}, inplace=True)
data

,order_id,quantity,item_name,choice_description,item_price($)
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


In [6]:
price_rank = data.groupby('item_name').mean()['item_price($)'].sort_values(ascending=False)
print(price_rank)
print("가장 비싼 음식은 {}로 {}이다".format(price_rank.iloc[0], price_rank.index[0]))

item_name
Bowl                                     14.800000
Steak Salad Bowl                         11.847931
Chicken Salad Bowl                       11.170455
Carnitas Crispy Tacos                    11.137143
Carnitas Salad Bowl                      11.056667
Barbacoa Crispy Tacos                    10.928182
Carnitas Bowl                            10.833971
Steak Bowl                               10.711801
Barbacoa Salad Bowl                      10.640000
Veggie Soft Tacos                        10.565714
Steak Burrito                            10.465842
Veggie Bowl                              10.211647
Steak Crispy Tacos                       10.209714
Barbacoa Bowl                            10.187273
Veggie Salad Bowl                        10.138889
Carnitas Burrito                         10.132712
Chicken Bowl                             10.113953
Chicken Burrito                          10.082857
Chicken Crispy Tacos                     10.045319
Barbacoa Soft Tacos  

### 3. 어떤 음식 조합이 제일 선호될까?

In [7]:
# 필요없는 column 삭제
data_t3 = data.drop(['order_id', 'item_price($)'], axis='columns')
data_t3

,quantity,item_name,choice_description
0,1,Chips and Fresh Tomato Salsa,NaN
1,1,Izze,[Clementine]
2,1,Nantucket Nectar,[Apple]
3,1,Chips and Tomatillo-Green Chili Salsa,NaN
4,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans..."
...,...,...,...
4617,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ..."
4618,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese..."
4619,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto..."
4620,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu..."


In [8]:
# choice_description 데이터 정제(결측값, 소스, 재료가 합쳐져있어서 구분해준다)
# choice_description에서 결측값(NaN)을 origin으로 설정한다
data_t3.fillna('Origin', inplace=True)
data_t3

,quantity,item_name,choice_description
0,1,Chips and Fresh Tomato Salsa,Origin
1,1,Izze,[Clementine]
2,1,Nantucket Nectar,[Apple]
3,1,Chips and Tomatillo-Green Chili Salsa,Origin
4,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans..."
...,...,...,...
4617,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ..."
4618,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese..."
4619,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto..."
4620,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu..."


In [9]:
# choice_description 은 [소스, [재료]] 형식으로 되어있음
data_t3.choice_description.unique()[:7]

array(['Origin', '[Clementine]', '[Apple]',
       '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]',
       '[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]',
       '[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]',
       '[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]'],
      dtype=object)

In [10]:
l = '[Tomatillo Red Chili Salsa, ]'
a = l.strip('[], ').split(',')
a

['Tomatillo Red Chili Salsa']

In [11]:
import re
p = re.compile('\[[^\[\]]*\]')

def saurce_ingredient(mix):
  if mix != 'Origin':
    ingredient_li = p.findall(mix)
    ingredient_li = ingredient_li[0][1:-1].split(',')
    ingredient_li = [s.strip() for s in ingredient_li]
    ingredient = ",".join(ingredient_li)
    if ingredient == '': ingredient = 'Origin'

    saurce = p.sub('', mix)
    saurce = saurce.strip('[], ')
    saurce_li = saurce.split(',')
    saurce_li = [s.strip() for s in saurce_li if s != '']
    saurce = ",".join(saurce_li)
    if saurce == '': saurce = 'Origin'

    return saurce, ingredient
  else : return 'Origin', 'Origin'

tt = '[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]'
saurce_ingredient(tt)

('Tomatillo Red Chili Salsa',
 'Fajita Vegetables,Black Beans,Pinto Beans,Cheese,Sour Cream,Guacamole,Lettuce')

In [12]:
data_t3['saurce'] = data_t3['choice_description'].map(lambda x : saurce_ingredient(x)[0])
data_t3['ingredient'] = data_t3['choice_description'].map(lambda x : saurce_ingredient(x)[1])
data_result = data_t3.drop(['choice_description'], axis=1)
data_result

,quantity,item_name,saurce,ingredient
0,1,Chips and Fresh Tomato Salsa,Origin,Origin
1,1,Izze,Origin,Clementine
2,1,Nantucket Nectar,Origin,Apple
3,1,Chips and Tomatillo-Green Chili Salsa,Origin,Origin
4,2,Chicken Bowl,Tomatillo-Red Chili Salsa (Hot),"Black Beans,Rice,Cheese,Sour Cream"
...,...,...,...,...
4617,1,Steak Burrito,Fresh Tomato Salsa,"Rice,Black Beans,Sour Cream,Cheese,Lettuce,Gua..."
4618,1,Steak Burrito,Fresh Tomato Salsa,"Rice,Sour Cream,Cheese,Lettuce,Guacamole"
4619,1,Chicken Salad Bowl,Fresh Tomato Salsa,"Fajita Vegetables,Pinto Beans,Guacamole,Lettuce"
4620,1,Chicken Salad Bowl,Fresh Tomato Salsa,"Fajita Vegetables,Lettuce"


In [13]:
total_num = len(data_result)
duplicated_num = len(data_result.drop_duplicates(['item_name', 'saurce', 'ingredient'], ignore_index=True))
print('중복 제거 전 : {}\n중복 제거 후 : {}'.format(total_num, duplicated_num))

중복 제거 전 : 4622
중복 제거 후 : 1796


In [14]:
result_tmp=data_result.groupby(['item_name','saurce'])['ingredient'].value_counts()
result_tmp=result_tmp.to_frame()
result_tmp

ingredient
item_name         saurce                            ingredient                                                    
6 Pack Soft Drink Origin                            Diet Coke                                                   24
                                                    Coke                                                        21
                                                    Sprite                                                       5
                                                    Lemonade                                                     2
                                                    Nestea                                                       2
...                                                                                                            ...
Veggie Soft Tacos Fresh Tomato Salsa (Mild)         Black Beans,Rice,Cheese,Lettuce                              1
                                                    Pinto Beans,Rice,Cheese,Sour Cream                           1
                  Roasted Chili Corn Salsa          Fajita Vegetables,Black Beans,Pinto Beans,Chees...           1
                  Roasted Chili Corn Salsa (Medium) Black Beans,Fajita Veggies,Cheese,Sour Cream,Le...           1
                  Tomatillo Red Chili Salsa         Fajita Vegetables,Rice,Black Beans,Cheese,Guaca...           1

[1796 rows x 1 columns]